In [ ]:
!pip install comet_ml --quiet

     |████████████████████████████████| 299 kB 10.9 MB/s 
     |████████████████████████████████| 54 kB 1.8 MB/s 
     |████████████████████████████████| 530 kB 36.2 MB/s 
     |████████████████████████████████| 52 kB 974 kB/s 


In [ ]:
import comet_ml

comet_ml.init(project_name="comet-optimizer")

Please enter your Comet API key from https://www.comet.ml/api/my/settings/
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Comet API key is valid
COMET INFO: Comet API key saved in /root/.comet.config


In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical


def build_model_graph(experiment):
    model = Sequential()
    model.add(
        Dense(
            experiment.get_parameter("first_layer_units"),
            activation="sigmoid",
            input_shape=(784,),
        )
    )
    model.add(Dense(128, activation="sigmoid"))
    model.add(Dense(128, activation="sigmoid"))
    model.add(Dense(10, activation="softmax"))
    model.compile(
        loss="categorical_crossentropy",
        optimizer=RMSprop(),
        metrics=["accuracy"],
    )
    return model


def train(experiment, model, x_train, y_train, x_test, y_test):
    model.fit(
        x_train,
        y_train,
        batch_size=experiment.get_parameter("batch_size"),
        epochs=experiment.get_parameter("epochs"),
        validation_data=(x_test, y_test),
    )


def evaluate(experiment, model, x_test, y_test):
    score = model.evaluate(x_test, y_test, verbose=0)
    print("Score %s", score)


def get_dataset():
    num_classes = 10

    # the data, shuffled and split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_train = x_train.reshape(60000, 784)
    x_test = x_test.reshape(10000, 784)
    x_train = x_train.astype("float32")
    x_test = x_test.astype("float32")
    x_train /= 255
    x_test /= 255
    print(x_train.shape[0], "train samples")
    print(x_test.shape[0], "test samples")

    # convert class vectors to binary class matrices
    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)

    return x_train, y_train, x_test, y_test


# Get the dataset:
x_train, y_train, x_test, y_test = get_dataset()

60000 train samples
10000 test samples


In [ ]:
# The optimization config:
config = {
    "algorithm": "bayes",
    "name": "Optimize MNIST Network",
    "spec": {"maxCombo": 5, "objective": "minimize", "metric": "loss"},
    "parameters": {
        "first_layer_units": {
            "type": "integer",
            "mu": 500,
            "sigma": 50,
            "scalingType": "normal",
        },
        "batch_size": {"type": "discrete", "values": [64, 128, 256]},
    },
    "trials": 1,
}

opt = comet_ml.Optimizer(config)

COMET INFO: COMET_OPTIMIZER_ID=5d81d806b4c947d2ae8741bb8c21c1c0
COMET INFO: Using optimizer config: {'algorithm': 'bayes', 'configSpaceSize': 'infinite', 'endTime': None, 'id': '5d81d806b4c947d2ae8741bb8c21c1c0', 'lastUpdateTime': None, 'maxCombo': 10, 'name': 'Optimize MNIST Network', 'parameters': {'batch_size': {'type': 'discrete', 'values': [64, 128, 256]}, 'first_layer_units': {'mu': 500, 'scalingType': 'normal', 'sigma': 50, 'type': 'integer'}}, 'predictor': None, 'spec': {'gridSize': 10, 'maxCombo': 10, 'metric': 'loss', 'minSampleSize': 100, 'objective': 'minimize', 'retryAssignLimit': 0, 'retryLimit': 1000}, 'startTime': 24021673633, 'state': {'mode': None, 'seed': None, 'sequence': [], 'sequence_i': 0, 'sequence_pid': None, 'sequence_retry': 0, 'sequence_retry_count': 0}, 'status': 'running', 'suggestion_count': 0, 'trials': 1, 'version': '2.0.1'}


In [ ]:
for experiment in opt.get_experiments():
    # Log parameters, or others:
    experiment.log_parameter("epochs", 10)

    # Build the model:
    model = build_model_graph(experiment)

    # Train it:
    train(experiment, model, x_train, y_train, x_test, y_test)

    # How well did it do?
    evaluate(experiment, model, x_test, y_test)

    # Optionally, end the experiment:
    experiment.end()